# Bibliotecas

In [ ]:
import pandas as pd

# Limpeza dos dados

## Lendo os arquivos

In [94]:
#produto = "smartphones"
#produto = "notebooks"
produto = "tvs"

In [95]:
df_americanas = pd.read_csv(f"Dados/Produtos/americanas_produtos_{produto}.csv", dtype = {'ean': str})
df_amazon = pd.read_csv(f"Dados/Produtos/amazon_produtos_{produto}.csv", dtype = {'ean': str})

In [96]:
df_amazon["ean"].value_counts()

7908414405448    2
7908414412934    2
7908414421097    1
0850013114054    1
7898571461398    1
                ..
7893299915956    1
0889028138366    1
7893299914379    1
0074000377233    1
7891356088599    1
Name: ean, Length: 246, dtype: int64

## Dropando Produtos sem EAN

In [97]:
print(f"Amazon: {df_amazon.shape}\nAmericanas: {df_americanas.shape}")

Amazon: (288, 6)
Americanas: (1524, 6)


In [98]:
df_americanas = df_americanas[df_americanas["ean"].notnull()]
df_amazon = df_amazon[df_amazon["ean"].notnull()]

In [ ]:
print(f"Amazon: {df_amazon.shape}\nAmericanas: {df_americanas.shape}")

## Adicionando a coluna do título com caixa baixa

In [99]:
df_amazon['titulo_cb'] = df_amazon['titulo'].str.lower()
df_americanas['titulo_cb'] = df_americanas['titulo'].str.lower()

## Retirando títulos com palavras indesejadas

In [100]:
df_americanas["descricao"] = df_americanas["descricao"].apply(str)
df_americanas = df_americanas[~df_americanas["descricao"].str.contains('Preço especial só pra essa semana!!')]

In [101]:
'''bad_words = ["acessório", "acessorio", "adaptador", "alargadores", "anel", "arduino", "atomiser", "avatar",
             "aparelho de som", "automotivo", "axiom",
             "bandeja", "biometria", "bike", "boroscópio", "box", 
             "cable", "cabo", "caixa", "campainha", "calculator", "calculadora", "capa", "capinha", "card",
             "case", "caso", "carregador", "carro"
             "chassi", "chaveiro", "chave seletora", "clip",
             "coletor", "collar", "conjunto", "comoda", "cômoda", "cordão", "cortador", "cosplay",
             "drone", "eon", "eixo", "eixos", "espelho", "filme", "franquia", "gancho", "grânulos", "gaveta",
             "handmade", "hemobblo", "hemobllo", "hd externo", "jbl", "junta", "kit",
             "lente", "lixa", "lurrose",
             "m-disc", "mágico", "magico", "mesa", "microsd", "monitor", "montagem", "moochy",
             "netcna", "óculos", "oculos", "ornament",
             "papel", "parede", "part", "partes", "pedal", "pen drive", "pendrive", "pneu", "pneus", "power bank",
             "prato", "prettyzoom", "pretyzoom", "proteção", "protetor", "pulseira",
             "raspberry", "radiador", "rank de memória", "realidade virtual", "refrigerador", "regeneradora",
             "relogio", "relógio", "remoção", "reparo", "reposição", "reposiçao", "resfriador", "retrovisor", "roda",
             "ronyme", "rv", "scanner", "sensor", "smart tv", "soquete", "stick", "streaming box", "suporte",
             "tablet", "tampa", "tomada", "tomadas", "tradutor", "turbopower",
             "ultra-sensível", "upgrade", "upgrades",
             "vador", "veado", "vedação", "verbatim", "vidro", "vr", "xilofone", "xylophone",
             "watch", "zawa", "zbox"
            ]'''

bad_words = ["caminhão", "caminhao", "onibus", "ônibus", "van", "lancha", "barco", "volkswagen", "motor home", "motorhome"]

print(f"Amazon: {df_amazon.shape}\nAmericanas: {df_americanas.shape}")

df_amazon_ft = df_amazon[~df_amazon["titulo_cb"].str.contains('|'.join(bad_words))]
df_americanas_ft = df_americanas[~df_americanas["titulo_cb"].str.contains('|'.join(bad_words))]
print(f"\nFiltrado:\nAmazon: {df_amazon_ft.shape}\nAmericanas: {df_americanas_ft.shape}")

Amazon: (248, 7)
Americanas: (428, 7)

Filtrado:
Amazon: (245, 7)
Americanas: (424, 7)


In [ ]:
#df_amazon_ft['url'].loc[970]

In [ ]:
#df_americanas_ft[350:].head(50)

## Encontrando EANs repetidos na mesma loja

### Removendo produtos com o mesmo título

In [102]:
df_amazon_ft = pd.DataFrame(df_amazon_ft.loc[~df_amazon_ft["titulo"].duplicated(keep = 'first')])
df_americanas_ft = pd.DataFrame(df_americanas_ft.loc[~df_americanas_ft["titulo"].duplicated(keep = 'first')])

In [103]:
print(f"Amazon: {df_amazon_ft.shape}\nAmericanas: {df_americanas_ft.shape}")

Amazon: (244, 7)
Americanas: (416, 7)


### Verificando Títulos de EANs duplicados

In [104]:
ama_vc_ean = df_amazon_ft[df_amazon_ft["ean"].notnull()]["ean"].value_counts()
ama_duplicados = ama_vc_ean[ama_vc_ean > 1].index[0]
df_amazon_ft[df_amazon_ft['ean'] == ama_duplicados]

,categoria,descricao,ean,preco,titulo,url,titulo_cb
141,"Eletrônicos e Tecnologia/TV, Áudio e Cinema em...",Conte com uma tela de 39'' HD de alta resoluçã...,7908414405448,NaN,Tela 39 Pol HD sem Conversor Digital Entradas ...,https://www.amazon.com.br/Tela-Multilaser-sem-...,tela 39 pol hd sem conversor digital entradas ...
142,"Eletrônicos e Tecnologia/TV, Áudio e Cinema em...",Conte com uma tela de 39'' HD de alta resoluçã...,7908414405448,NaN,"Tela Multilaser 39"" HD sem Conversor Digital -...",https://www.amazon.com.br/Tela-Multilaser-sem-...,"tela multilaser 39"" hd sem conversor digital -..."


In [105]:
ame_vc_ean = df_americanas_ft[df_americanas_ft["ean"].notnull()]["ean"].value_counts()
ame_duplicados = ame_vc_ean[ame_vc_ean > 1].index
df_americanas_ft[df_americanas_ft["ean"].isin(ame_duplicados)].sort_values(by = ['ean']).head(1)

,categoria,descricao,ean,preco,titulo,url,titulo_cb
1432,/categoria/tv-e-home-theater/tv,atenção: Esse produto não pode ser faturado pa...,7891356081873,"R$ 1.546,90",Tv Ptv24g50sn Led Bivolt,https://www.americanas.com.br/produto/3208276209,tv ptv24g50sn led bivolt


In [106]:
df_americanas_ft['loja'] = "americanas"
df_amazon_ft['loja'] = "amazon"

In [107]:
frames = [df_americanas_ft, df_amazon_ft]
df_final = pd.concat(frames, ignore_index = True)

In [108]:
vc_ean = df_final[df_final["ean"].notnull()]["ean"].value_counts()
duplicados = vc_ean[vc_ean > 1].index
df_final_dup = df_final[df_final["ean"].isin(duplicados)].sort_values(by = ['ean'])

## Fazendo matches

In [109]:
def criar_dicionario(descricao_1, ean_1, titulo_1, url_1, titulo_cb_1, loja_1,
                     descricao_2, ean_2, titulo_2, url_2, titulo_cb_2, loja_2,
                     match):
    
    return {'descricao_1' : descricao_1 , 'ean_1' : ean_1, 'titulo_1' : titulo_1,
            'url_1' : url_1, 'titulo_cb_1' : titulo_cb_1, 'loja_1' : loja_1,
            
            'descricao_2' : descricao_2 , 'ean_2' : ean_2, 'titulo_2' : titulo_2,
            'url_2' : url_2 , 'titulo_cb_2' : titulo_cb_2, 'loja_2' : loja_2,
            
            'match' : match
           }

In [110]:
def criar_pares(colunas, df_final_dup):

    df_pares = pd.DataFrame(columns = colunas)
    df_pares

    lista_ean = []

    for i in range(len(df_final_dup)):
        
        if df_final_dup.iloc[i]['ean'] not in lista_ean:
            
            lista_ean.append(df_final_dup.iloc[i]['ean'])
            
            indice = df_final_dup.index[df_final_dup['ean'] == df_final_dup.iloc[i]['ean']].tolist()
            
            for j in range(len(indice)-1):
                
                for k in range(j + 1, len(indice)):
                    
                    dicionario = criar_dicionario(descricao_1  = df_final_dup.loc[indice[j]]['descricao'] , ean_1 = df_final_dup.loc[indice[j]]['ean'],
                                                  titulo_1 = df_final_dup.loc[indice[j]]['titulo'], url_1 = df_final_dup.loc[indice[j]]['url'],
                                                  titulo_cb_1 = df_final_dup.loc[indice[j]]['titulo_cb'], loja_1 = df_final_dup.loc[indice[j]]['loja'],
                                                  
                                                  descricao_2 = df_final_dup.loc[indice[k]]['descricao'] , ean_2 = df_final_dup.loc[indice[k]]['ean'],
                                                  titulo_2 = df_final_dup.loc[indice[k]]['titulo'], url_2 = df_final_dup.loc[indice[k]]['url'],
                                                  titulo_cb_2 = df_final_dup.loc[indice[k]]['titulo_cb'], loja_2 = df_final_dup.loc[indice[k]]['loja'],
                                                  
                                                  match = 1
                                                  )
                
                    df_pares = df_pares.append(dicionario, ignore_index = True)

    return df_pares

In [111]:
colunas = ["descricao_1", "ean_1", "titulo_1", "url_1",	"titulo_cb_1", "loja_1",
           "descricao_2", "ean_2", "titulo_2", "url_2",	"titulo_cb_2", "loja_2",
           "match"]

In [112]:
df_pares = criar_pares(colunas, df_final_dup)

C:\Users\Amamiya\AppData\Local\Temp\ipykernel_15056\3485444683.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_pares = df_pares.append(dicionario, ignore_index = True)
C:\Users\Amamiya\AppData\Local\Temp\ipykernel_15056\3485444683.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_pares = df_pares.append(dicionario, ignore_index = True)
C:\Users\Amamiya\AppData\Local\Temp\ipykernel_15056\3485444683.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_pares = df_pares.append(dicionario, ignore_index = True)
C:\Users\Amamiya\AppData\Local\Temp\ipykernel_15056\3485444683.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  

In [ ]:
#df_pares[df_pares['ean_1']=="6934177738180"][['descricao_1', 'ean_1', 'titulo_1', 'descricao_2', 'ean_2', 'titulo_2', 'loja_1', 'loja_2', 'url_1', 'url_2']]
#df_pares[['descricao_1', 'ean_1', 'titulo_1', 'descricao_2', 'ean_2', 'titulo_2', 'loja_1', 'loja_2', 'url_1', 'url_2']]

In [113]:
df_pares.shape

(327, 13)

In [114]:
df_pares

,descricao_1,ean_1,titulo_1,url_1,titulo_cb_1,loja_1,descricao_2,ean_2,titulo_2,url_2,titulo_cb_2,loja_2,match
0,Televisor com tela de 24 polegadas em resoluçã...,7891356081873,"Smart Tv Philco 24"" Resolução Em Hd Fast Smart...",https://www.americanas.com.br/produto/1803456440,"smart tv philco 24"" resolução em hd fast smart...",americanas,atenção: Esse produto não pode ser faturado pa...,7891356081873,Tv Ptv24g50sn Led Bivolt,https://www.americanas.com.br/produto/3208276209,tv ptv24g50sn led bivolt,americanas,1
1,Televisor com tela de 39 polegadas em resoluçã...,7891356086595,TV LED 39'' Philco PTV39G50D Resolução HD e Re...,https://www.americanas.com.br/produto/1695817631,tv led 39'' philco ptv39g50d resolução hd e re...,americanas,"TV Philco PTV39G50D d-led 39""",7891356086595,TV Philco PTV39G50D d-led 39,https://www.americanas.com.br/produto/4660650164,tv philco ptv39g50d d-led 39,americanas,1
2,Televisor com tela de 39 polegadas em resoluçã...,7891356086595,TV LED 39'' Philco PTV39G50D Resolução HD e Re...,https://www.americanas.com.br/produto/1695817631,tv led 39'' philco ptv39g50d resolução hd e re...,americanas,atenção: Esse produto não pode ser faturado pa...,7891356086595,Tv Philco 39 PTV39G50D Led Digital Bivolt,https://www.americanas.com.br/produto/3361353135,tv philco 39 ptv39g50d led digital bivolt,americanas,1
3,"TV Philco PTV39G50D d-led 39""",7891356086595,TV Philco PTV39G50D d-led 39,https://www.americanas.com.br/produto/4660650164,tv philco ptv39g50d d-led 39,americanas,atenção: Esse produto não pode ser faturado pa...,7891356086595,Tv Philco 39 PTV39G50D Led Digital Bivolt,https://www.americanas.com.br/produto/3361353135,tv philco 39 ptv39g50d led digital bivolt,americanas,1
4,TV PTV32G70SBL LED Smart Philco Possui WIFI in...,7891356088599,Smart TV Philco PTV32G70SBL LED- HD- WIFI inte...,https://www.amazon.com.br/Smart-Philco-PTV32G7...,smart tv philco ptv32g70sbl led- hd- wifi inte...,amazon,Televisor com tela de 32 polegadas em resoluçã...,7891356088599,"Smart Tv Philco 32"" Em Led Hd. Áudio Dolby, Co...",https://www.americanas.com.br/produto/1803603420,"smart tv philco 32"" em led hd. áudio dolby, co...",americanas,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
322,Com a Tela HD de 24 Polegadas Smart da Multila...,7908414430679,TV Tela 24 Polegadas HD com Função Smart e Wif...,https://www.amazon.com.br/Polegadas-Fun%C3%A7%...,tv tela 24 polegadas hd com função smart e wif...,amazon,Informações do produto Com a Tela HD de 24 Pol...,7908414430679,Tv 24 Multilaser Tl040 - Smart Tv - Hd - Wi-Fi...,https://www.americanas.com.br/produto/4817515287,tv 24 multilaser tl040 - smart tv - hd - wi-fi...,americanas,1
323,Descrição resumidaConte com uma tela de 42 HD ...,7908414437388,Televisao 42 Polegadas Fhd Com Funcao Smart E ...,https://www.americanas.com.br/produto/4796186313,televisao 42 polegadas fhd com funcao smart e ...,americanas,Conte com uma tela Full HD Smart de 42” de alt...,7908414437388,TV Tela 42 Polegadas FHD com Função Smart e Wi...,https://www.amazon.com.br/Polegadas-Fun%C3%A7%...,tv tela 42 polegadas fhd com função smart e wi...,amazon,1
324,Todas as informações divulgadas são de respons...,8806090491559,"Samsung Smart Tv Qled 4k The Sero 2020 43"", Tv...",https://www.americanas.com.br/produto/1829677970,"samsung smart tv qled 4k the sero 2020 43"", tv...",americanas,Modelo: LS05TAGXZD O entretenimento do seu sma...,8806090491559,Samsung Smart TV qled 4K LS05T The Sero 2020 T...,https://www.americanas.com.br/produto/2154521642,samsung smart tv qled 4k ls05t the sero 2020 t...,americanas,1
325,Todas as informações divulgadas são de respons...,8806090491559,"Samsung Smart Tv Qled 4k The Sero 2020 43"", Tv...",https://www.americanas.com.br/produto/1829677970,"samsung smart tv qled 4k the sero 2020 43"", tv...",americanas,NaN,8806090491559,"Samsung SmartTV QLED 4K The Sero2020 43, TV Ve...",https://www.amazon.com.br/Samsung-SmartTV-Sero...,"samsung smarttv qled 4k the sero2020 43, tv ve.

## Salvando Pares

In [115]:
df_pares.to_csv(f"Dados/Pares/pares_{produto}.csv", index = False)

In [116]:
df_pares = pd.read_csv(f"Dados/Pares/pares_{produto}.csv")

## Criando pares que não são matches

In [117]:
df_final_ean = df_final[df_final["ean"].notnull()]
df_final_ean.shape

(660, 8)

In [118]:
n_matches = df_pares.shape[0]*5
df_nao_pares = pd.DataFrame(columns = colunas)

for i in range(n_matches):
    created_match = False
    
    while(not created_match):
        df_sorteado = df_final_ean.sample(n = 2)
        if(df_sorteado.iloc[0]['ean']!= df_sorteado.iloc[1]['ean']):
            
            dic = criar_dicionario(df_sorteado.iloc[0]['descricao'], df_sorteado.iloc[0]['ean'], df_sorteado.iloc[0]['titulo'], df_sorteado.iloc[0]['url'], df_sorteado.iloc[0]['titulo_cb'], df_sorteado.iloc[0]['loja'],
                                   df_sorteado.iloc[1]['descricao'], df_sorteado.iloc[1]['ean'], df_sorteado.iloc[1]['titulo'], df_sorteado.iloc[1]['url'], df_sorteado.iloc[1]['titulo_cb'], df_sorteado.iloc[1]['loja'],
                                   0)
            
            df_nao_pares = df_nao_pares.append(dic, ignore_index = True)
            created_match = True

C:\Users\Amamiya\AppData\Local\Temp\ipykernel_15056\2934401280.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_nao_pares = df_nao_pares.append(dic, ignore_index = True)
C:\Users\Amamiya\AppData\Local\Temp\ipykernel_15056\2934401280.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_nao_pares = df_nao_pares.append(dic, ignore_index = True)
C:\Users\Amamiya\AppData\Local\Temp\ipykernel_15056\2934401280.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_nao_pares = df_nao_pares.append(dic, ignore_index = True)
C:\Users\Amamiya\AppData\Local\Temp\ipykernel_15056\2934401280.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

Salvando pares não matches

In [119]:
df_nao_pares.to_csv(f"Dados/Não Pares/nao_pares_{produto}.csv", index = False)

## Gerando dataset final

In [120]:
df_pares = pd.read_csv(f"Dados/Pares/pares_{produto}.csv")
df_nao_pares = pd.read_csv(f"Dados/Não Pares/nao_pares_{produto}.csv")

df_pares_total = pd.concat([df_pares, df_nao_pares], ignore_index = True)

In [121]:
df_pares_total = df_pares_total.sample(frac = 1)
df_pares_total.to_csv(f"Dados/Total Pares/total_pares_{produto}.csv", index = False)

In [125]:
df_pares_total.shape

(1962, 13)

In [122]:
df_pares_total

,descricao_1,ean_1,titulo_1,url_1,titulo_cb_1,loja_1,descricao_2,ean_2,titulo_2,url_2,titulo_cb_2,loja_2,match
1195,Já pensou em uma TV com tela infinita? A Smart...,7892509113472,"Smart TV 4K Samsung 55 uhd, 2 hdmi, 1 usb, Wi-...",https://www.americanas.com.br/produto/2474000256,"smart tv 4k samsung 55 uhd, 2 hdmi, 1 usb, wi-...",americanas,"Conte com uma tela de 39 HD de alta resolução,...",7908414405271,Tela 39 polegadas HD Multilaser - TL028,https://www.americanas.com.br/produto/4522563089,tela 39 polegadas hd multilaser - tl028,americanas,0
733,Recursos:,8130884729512,"Monitor, Tela sensível ao toque de 10,1 polega...",https://www.amazon.com.br/sens%C3%ADvel-polega...,"monitor, tela sensível ao toque de 10,1 polega...",amazon,"Smart TV LG 55"" 4K NanoCell 55NANO80 4x HDMI 2...",7893299915772,"2021 Smart TV LG 55"" 4K NanoCell 55NANO80 4x H...",https://www.amazon.com.br/LG-NanoCell-55NANO80...,"2021 smart tv lg 55"" 4k nanocell 55nano80 4x h...",amazon,0
1372,A Smart TV LED 50 Ultra HD 4K da Multilaser pr...,7899838892825,Smart Tv 50 Polegadas Led 4k Wifi Integrado Co...,https://www.americanas.com.br/produto/2628501476,smart tv 50 polegadas led 4k wifi integrado co...,americanas,Característica:,7043187117867,Zopsc-1 Bateria recarregável portátil de TV de...,https://www.amazon.com.br/Zopsc-1-recarreg%C3%...,zopsc-1 bateria recarregável portátil de tv de...,amazon,0
1,Televisor com tela de 39 polegadas em resoluçã...,7891356086595,TV LED 39'' Philco PTV39G50D Resolução HD e Re...,https://www.americanas.com.br/produto/1695817631,tv led 39'' philco ptv39g50d resolução hd e re...,americanas,"TV Philco PTV39G50D d-led 39""",7891356086595,TV Philco PTV39G50D d-led 39,https://www.americanas.com.br/produto/4660650164,tv philco ptv39g50d d-led 39,americanas,1
336,"Com a Smat TV Samsung Q60A, você aproveita não...",7892509116718,Smart TV Samsung QLED 4K 65Q60A Design slim Mo...,https://www.amazon.com.br/Samsung-65Q60A-Desig...,smart tv samsung qled 4k 65q60a design slim mo...,amazon,Todas as informações divulgadas são de respons...,7893299916960,"Smart TV OLED 83"" LG OLED83C1 4K 120hz G-Sync ...",https://www.americanas.com.br/produto/3849625667,"smart tv oled 83"" lg oled83c1 4k 120hz g-sync ...",americanas,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1811,Descubra o que há de mais novo em tecnologia d...,7892509117913,"Smart TV Samsung Led 55'' uhd, 4K, Alexa Built...",https://www.americanas.com.br/produto/4395177723,"smart tv samsung led 55'' uhd, 4k, alexa built...",americanas,"A Smart TV Ultra HD 4K LED de 55"" da Samsung é...",7892509118149,Smart TV LED 55 Samsung Ultra HD 4K hdr c/conv...,https://www.americanas.com.br/produto/4284341949,smart tv led 55 samsung ultra hd 4k hdr c/conv...,americanas,0
54,Conecta teus dispositivos por meio de seus 2 p...,7892509114233,Smart TV LED 32'' HD Samsung LH32BETBLGGXZD,https://www.amazon.com.br/Smart-TV-LED-Samsung...,smart tv led 32'' hd samsung lh32betblggxzd,amazon,"A Smart TV LED 32"" Samsung LH32BETBLGGXZD é eq...",7892509114233,"Samsung Smart TV 32"" LH32BETBLGGXZD LED 2 HDMI...",https://www.americanas.com.br/produto/1943745999,"samsung smart tv 32"" lh32betblggxzd led 2 hdmi...",americanas,1
1629,Características do Produto: Modelo: OLED55A1PS...,7893299916014,"Smart TV oled 55 LG OLED55A1PSA, 4K, Wi-Fi, Bl...",https://www.americanas.com.br/produto/4880314074,"smart tv oled 55 lg oled55a1psa, 4k, wi-fi, bl...",americanas,Smart TV Philco 65'' 4K Led PTV65G70AGCBL – Bi...,7891356100352,Smart TV Philco 65'' 4K Led PTV65G70AGCBL – Bi...,https://www.americanas.com.br/produto/4156323228,smart tv philco 65'' 4k led ptv65g70agcbl – bi...,americanas,0
359,Descubra uma emocionante experiência e uma man...,7899968300863,Smart Tv LED 32 Hdmi Wi-fi Usb - Semp,https://www.americanas.com.br/produto/4251300753,smart tv led 32 hdmi wi-fi usb - semp,americanas,Todas as informações divulgadas são de respons...,7893299915901,Smart TV LED 75” LG 75NANO75 4K NanoCell 3x HD...,https://www.ame